In [ ]:
from sklearn.utils import resample

In [ ]:
# Separate minority and majority classes
win = df_completed_matches[df_completed_matches['Prediction Result(won/loss)']==0]
loss = df_completed_matches[df_completed_matches['Prediction Result(won/loss)']==1]

In [ ]:
import matplotlib.pyplot as plt
labels = df_completed_matches['Prediction Result(won/loss)'].unique()
heights = [len(win),len(loss)]
plt.bar(labels,heights,color='grey')
plt.xticks(labels,['win','loss'])
plt.ylabel("# of observations")
plt.show()

In [ ]:
loss_upsampled = resample(loss,
                          replace=True, # sample with replacement (we need to duplicate observations)
                          n_samples=len(win), # match number in minority class
                          random_state=27) # reproducible results

# Combine upsampled minority class with majority class
upsampled = pd.concat([loss_upsampled, win])

# Check new class counts
upsampled['Prediction Result(won/loss)'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


X = loss_upsampled.drop(columns=['Prediction Result(won/loss)', 'Weather in degrees','Full time score','Score at halftime', 'Date','Time'])
# Define your target variable y
y = df_completed_matches['Prediction Result(won/loss)']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the logistic regression model
log_reg = LogisticRegression(max_iter=10000, solver='sag')

# Train the model on the training data
log_reg.fit(X_train, y_train)

# Predict the target variable on the testing set
y_pred = log_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Get more detailed classification report
print(classification_report(y_test, y_pred))